In [1]:
import ee
ee.Authenticate()
# Initialize Earth Engine
ee.Initialize()

In [4]:
import ee
import datetime

# Define an area of interesrt (Mecca + 10 km buffer)
mecca = ee.Geometry.Point([39.826, 21.422])
buffer = mecca.buffer(40000)

# Load Landsat 8/9 Surface Temperature (Collection 2 Level-2) --I think i
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterBounds(buffer) 

# Function to compute LST (°C) and mask clouds
def compute_lst(img):
    lst = img.select("ST_B10").multiply(0.00341802).add(149).subtract(273.15).rename("LST_Celsius")
    qa = img.select("QA_PIXEL")
    cloud_mask = qa.bitwiseAnd(1 << 3).eq(0) \
        .And(qa.bitwiseAnd(1 << 4).eq(0)) \
        .And(qa.bitwiseAnd(1 << 5).eq(0))
    return lst.updateMask(cloud_mask).copyProperties(img, ["system:time_start"])

lst_collection = landsat.map(compute_lst)

Export started for 20130412
Export started for 20130428
Export started for 20130514
Export started for 20130530
Export started for 20130615
Export started for 20130701
Export started for 20130802
Export started for 20130818
Export started for 20130903
Export started for 20130919
Export started for 20131005
Export started for 20131021
Export started for 20131106
Export started for 20131122
Export started for 20131208
Export started for 20131224
Export started for 20130419
Export started for 20130505
Export started for 20130521
Export started for 20130606
Export started for 20130622
Export started for 20130708
Export started for 20130724
Export started for 20130809
Export started for 20130825
Export started for 20130910
Export started for 20130926
Export started for 20131012
Export started for 20131028
Export started for 20131113
Export started for 20131129
Export started for 20131215
Export started for 20140109
Export started for 20140125
Export started for 20140210
Export started for 2

In [ ]:
# Split by year and export
start_year = 2013
end_year = 2024

for year in range(start_year, end_year + 1):
    start = ee.Date(f"{year}-01-01")
    end = ee.Date(f"{year}-12-31")
    
    yearly_collection = lst_collection.filterDate(start, end)

    def export_image(img):
        date_str = ee.Date(img.get("system:time_start")).format("YYYYMMdd").getInfo()
        task = ee.batch.Export.image.toDrive(
            image=img.clip(buffer),
            description=f"LST_Mecca_{date_str}",
            folder="GEE_LST_TIMESERIES",
            fileNamePrefix=f"Mecca_LST_{date_str}",
            region=buffer,
            scale=30,
            maxPixels=1e13
        )
        task.start()
        print(f"Export started for {date_str}")

    # Export all images for the year
    images = yearly_collection.toList(yearly_collection.size())
    for i in range(images.size().getInfo()):
        export_image(ee.Image(images.get(i)))

In [ ]:
import time

# Define your desired sleep time in seconds (e.g., 10 seconds)
sleep_time = 10  

start_year = 1984
end_year = 2024

for year in range(start_year, end_year + 1):
    start = ee.Date(f"{year}-01-01")
    end = ee.Date(f"{year}-12-31")
    
    yearly_collection = landsat_indexed.filterDate(start, end)

    def export_image(img):
        date_str = ee.Date(img.get("system:time_start")).format("YYYYMMdd").getInfo()
        task = ee.batch.Export.image.toDrive(
            image=img.clip(bbox),
            description=f"LST_Mecca_{date_str}",
            folder="GEE_LST_TIMESERIES",
            fileNamePrefix=f"Mecca_LST_{date_str}",
            region=bbox,
            scale=30,
            maxPixels=1e20
        )
        task.start()
        print(f"Export started for {date_str}")
        time.sleep(sleep_time)  # Pause to reduce risk of API throttling

    images = yearly_collection.toList(yearly_collection.size())
    num_images = images.size().getInfo()

    for i in range(num_images):
        try:
            img = ee.Image(images.get(i))
            export_image(img)
        except Exception as e:
            print(f"Export failed for image {i}: {e}")
